#  US Gap Analysis Project - WV Breeding Bird Atlas Data Comparison 
Nathan Tarr and Jessie Jordan

## Elevation
We investigated the agreement between data collected by the WV Breeding Bird Atlas (2011-2015) and GAP's deductive habitat models and 2001 habitat maps.  Here, we present the results comparisons of elevation information.

We compared three things: the elevations of survey locations where individuals were recorded by the WVBBA, the maximum and minimum elevations of habitat grid cells as mapped by GAP for each species, and the elevation parameter values from GAP habitat models.  We obtained WVBBA data from WV biologists who ran that project.  We determined the maximum and minimum elevations of habitat grid cells by summarizing their elevations with the arcpy package ("map_max" and "map_min").  Habitat model parameters were retrieved from ScienceBase items for the models.  The border between GAP's southeastern and northeastern modeling regions passes through WV, so habitat maps within WV could have been produced in part from two potentially different models.  We therefore used southeastern and northeastern model parameters in our comparisons ("NE_max", "NE_min", "SE_max", and "SE_min"), which, in some cases, complicated comparisons.

To quantify the agreement between data sets, we tallied the number of individuals that the WVBBA recorded that were above and below the max elevations from habitat maps ("individuals_over(n)" and "individuals_under(n)"), as well as the differences between maximum and minimum elevations ("max_diff" and "min_diff").  

In [1]:
import pandas as pd, repo_functions as fun, numpy as np
pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth', 400)
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', 15)

# Elevation summary csv
eDF0 = pd.read_csv(fun.resultsDir + "elevation_summary.csv", header=0, names=["GAP_code", "WVBBA_code", "common_name",
                                                                              "map_max", "map_min", "SE_max", "SE_min", 
                                                                              "NE_max", "NE_min", "WVBBA_max", "WVBBA_min",
                                                                              "max_diff(%)", "min_diff(%)", "scientific_name",
                                                                              "max_diff", "min_diff", "WVBBA_individuals(n)", 
                                                                              "individuals_over(n)", "individuals_under(n)"])
#eDF0.replace(to_replace='NULL', value=np.nan, inplace=True)
print("A sample of values\n")
eDF0.drop(["min_diff(%)","max_diff(%)"], axis=1, inplace=True)
eDF1 = eDF0.copy()
eDF0.set_index("GAP_code", inplace=True)
print(eDF0.head().T)

A sample of values

GAP_code                          bcogrx               bcoyex             bhowrx                 brbnux           brevix
WVBBA_code                          COGR                 COYE               HOWR                   RBNU             REVI
common_name               Common Grackle  Common Yellowthroat         House Wren  Red-breasted Nuthatch   Red-eyed Vireo
map_max                             1370                 1482               1480                   1482             1477
map_min                               75                   75                 77                     86               77
SE_max                              1371                  NaN                NaN                   2500              NaN
SE_min                                 0                  NaN                NaN                   1000              NaN
NE_max                              1371                  NaN                NaN                   2500              NaN
NE_min      

A complication existed regarding the elevations of WVBBA survey data.  Observers used a point count protocol in which bird detections were attributed to the survey point.  Although the survey location is an x,y point, the area surveyed is actually a polygon approximating a circle.  Thus, birds could have been detected from within a range of elevations despite being recorded at an x,y location.  The range of elevations surveyed is determined by the width of the circle that was effectively surveyed and the terrain.  The width of the circle and the distance at which individuals could be detected that ultimately determines it is affected by local factors including vegetation structure.  Thus, the elevation ranges assocated with detections varies among points  If the survey area is large and the terrain steep, then the range of elevations at which birds could have been detected is large.  Without knowing the conditions at each survey location, it is impossible to precisely attribute each record to an elevation, but ignoring this issue would introduce error in our assessment, therefore we approximated a margin of error of 75m for elevation values that we used when comparing WVBBA records to GAP data: values were considered equal when they were within 75 m of each other.  We chose 75m because individuals can often be heard from distance of 100m or more in auditory bird surveys and WV, in general, is mountainous.

In [2]:
# Set a margin of error for elevation
elev_error = 75

### Which models have innapropriate GAP max elevation parameters that should be raised for WV?
Cases where the maximum elevation parameter should be raised can be identified by determining which species were detected in significant frequency at elevations above the maximum elevation from the habitat map *and* for which the maximum from the map equals the maximum elevation parameter in the model.  It is possible for the maximum elevation parameter to be different from the map maximum when other model parameters, such as cover type selections, limit output to lower elevations.  Defining a "significant frequency" of WVBBA occurrences that defy the model parameter is subjective; we used a value of 1% of total detections.  Additionally, we applied the margin of error discussed above so that at least one detection had to be more than 75 m above the GAP maximum in order for frequency of errors to be assessed. 

In [19]:
# Which species were detected above max elevation from habitat maps?
df0 = eDF1[eDF1["individuals_over(n)"] > 0]
overMax = df0.filter(items=['GAP_code', 'common_name', 'individuals_over(n)', 'WVBBA_max', 'map_max', 'max_diff', 'SE_max', 
                            'NE_max', 'WVBBA_individuals(n)'], axis=1)
''' 
Omit records with difference greater than the set "elev_error" bcs. count circles have range of elevations and gps precision, 
which creates reason to question a conflict bcs. elev is 'close' to gap max.
'''
overMax = overMax[overMax['max_diff'] > elev_error]

'''
If only a small percentage of the detected individuals conflict with the map, then it's probably not appropriate to change the 
model, so filter out species w/ <1% of individuals conflicting with the model'
'''
overMax['individuals_over(%)'] = 100*(overMax['individuals_over(n)']/overMax['WVBBA_individuals(n)'])
overMax.sort_values(by="individuals_over(n)", ascending=False, inplace=True)
overMax = overMax[overMax['individuals_over(%)'] > 1]

# For which species was a model responsible and needs to be adjusted?
'''
If one of the model max's is NULL, then the map output max wasn't necessarily constrained by it, so exclude it.  
'''
overMax2 = overMax.copy().dropna()
overMax2.drop(["WVBBA_individuals(n)"], axis=1, inplace=True)
overMax2.set_index(["GAP_code"], inplace=True)
print("\n\nMax elevation needs to be adjusted in the models for these species")
print(overMax2.filter(["common_name", "individuals_over(n)", "individuals_over(%)", "max_diff", "WVBBA_max", "map_max", 
              "SE_max", "NE_max"], axis=1))



Max elevation needs to be adjusted in the models for these species
                     common_name  individuals_over(n)  individuals_over(%)  max_diff  WVBBA_max  map_max  SE_max  NE_max
GAP_code                                                                                                                
bchspx          Chipping Sparrow                  136            16.229117       387       1300      913   914.0   914.0
brbwox    Red-bellied Woodpecker                   26             3.576341       361       1260      899   900.0   900.0
bwiflx         Willow Flycatcher                   16            16.494845       371       1220      849   850.0   850.0
bororx            Orchard Oriole                   13            10.569106       331        940      609   610.0   610.0
bnoflx          Northern Flicker                    8             1.731602       122       1340     1218  1219.0  1219.0
bbekix         Belted Kingfisher                    8             9.302326       267

### For which species is something other than the elevation parameter restricting output to lower elevations?
Significant numbers of detections above the GAP map maximum elevation when the map max doesn't equal the model max indicates that something other than the model parameter restricted the habitat map to erroneously low elevations.

In [22]:
# For which species was something other than a model responsible for the error?
overMax3 = overMax[overMax['GAP_code'].isin(overMax2.index) == False].copy()
overMax3.drop(["WVBBA_individuals(n)"], axis=1, inplace=True)
overMax3.set_index("GAP_code", inplace=True)
print("\n\nSomething other than a max elevation parameter could be erroneously restricting map output to low elevations.")
print(overMax3.filter(["common_name", "individuals_over(n)", "individuals_over(%)", "max_diff", "WVBBA_max", "map_max", 
              "SE_max", "NE_max"], axis=1))



Something other than a max elevation parameter could be erroneously restricting map output to low elevations.
             common_name  individuals_over(n)  individuals_over(%)  max_diff  WVBBA_max  map_max  SE_max  NE_max
GAP_code                                                                                                        
beaphx    Eastern Phoebe                  202            38.403042       714       1160      446     NaN     NaN


In [21]:
# Which species were detected below min elevation from habitat maps?
df1 = eDF0[eDF0["individuals_under(n)"] > 0]
overMin = df1.filter(items=['GAP_code', 'common_name', 'individuals_under(n)', 
                            'WVBBA_min', 'map_min', 'min_diff', 'SE_min', 
                            'NE_min', 'WVBBA_individuals(n)'], 
                     axis=1)
''' 
Omit records with difference greater than the set "elev_error" bcs. count 
circles have range of elevations and gps precision, which creates reason to 
question a conflict bcs. elev is 'close' to gap min.
'''
overMin = overMin[overMin['min_diff'] > elev_error]

'''
If only a small percentage of the detected individuals conflict with the map,
then it's probably not appropriate to change the model, so filter out
species w/ <1% of individuals conflicting with the model
'''
overMin['individuals_under(%)'] = 100*(overMin['individuals_under(n)']/overMin['WVBBA_individuals(n)'])
overMin.sort_values(by="individuals_under(n)", ascending=False, inplace=True)
overMin = overMin[overMin['individuals_under(%)'] > 1]

# For which species was a model responsible and needs to be adjusted?
'''
If one of the model min's is NULL, then map output min wasn't necessarily
constrained by it, so exlude.  
'''
overMin2 = overMin.copy().dropna()
print("\n\nMin elevation needs to be adjusted in the models for these species")
print(overMin2)

# For which species was something other than a model responsible for the error?
overMin3 = overMin[overMin['GAP_code'].isin(overMin2['GAP_code']) == False]
print("\n\nSomething other than a min elevation parameter could be erroneously restricting map output to high elevations.")
print(overMin3)






Min elevation needs to be adjusted in the models for these species
                common_name  individuals_under(n)  WVBBA_min  map_min  min_diff  SE_min  NE_min  WVBBA_individuals(n)  individuals_under(%)
GAP_code                                                                                                                                   
bbhvix    Blue-headed Vireo                    24        220      301        81   300.0   300.0                   490              4.897959


KeyError: 'GAP_code'

In [ ]:
#################  OBSOLETE ELEVATION PARAMETERS  ############################
##############################################################################
# For which species is max map elevation less than the max elevation parameter
misMax = eDF0.copy().filter(items=['GAP_code', 'common_name', 'map_max', 'SE_max', 
                            'NE_max'], axis=1)
misMax = misMax.dropna()
misMax2 = misMax[(misMax['map_max'] + 1 < misMax['SE_max']) & (misMax['map_max'] + 1 < misMax['NE_max'])]
print("\n\nThese species have obsolete maximum elevations for WV")
print(misMax2)

misMin = eDF0.copy().filter(items=['GAP_code', 'common_name', 'map_min', 'SE_min', 
                            'NE_min'], axis=1)
misMin = misMin.dropna()
misMin2 = misMin[(misMin['map_min'] - 1 > misMin['SE_min']) & (misMin['map_min'] - 1 > misMin['NE_min'])]
print("\n\nThese species have obsolete minimum elevations for WV")
print(misMin2)



## Conclusions
Modeling regions prohibit model quality
